In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import h5py
import sys
import getopt
import os
import random

from all_utils import *

import matplotlib.pyplot as plt 
from distinct_colours import get_distinct#,cm_linear,cm_plusmin

from readsnap import readsnap

/software/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
a = np.linspace(0,10,11)
b = np.linspace(10,20,11)

a = np.array(a,dtype='f',ndmin=1)
b = np.array(b,dtype='f',ndmin=1)


In [3]:
import ctypes

In [17]:
exec_call = "/home/abg6257/CFoF/fof_sne.so"
c_obj = ctypes.CDLL(exec_call)

In [11]:
exec_call = "/home/abg6257/CFoF/fof_sne.so"
c_obj = ctypes.CDLL(exec_call)

In [12]:
h_out_cast=ctypes.c_float*11
H_OUT=h_out_cast()

In [13]:
c_obj.add_arrays(
    ctypes.c_int(11),
    a.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
    b.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
    ctypes.byref(H_OUT))

0

In [14]:
h=np.ctypeslib.as_array(H_OUT)
print h

[10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30.]
